# Investments Project (Spring 2024)

**Authors:**
- Marc-Antoine Allard
- Adam Zinebi
- Paul Teiletche
- ...

**DUE Date: June 21 at 23:59**

---
# Imports

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from utils import annualized_metrics

ImportError: cannot import name 'annualized_metrics' from 'utils' (/Users/paul/Desktop/Investments/Investments_project2024/source/utils.py)

---
# 4 - Momentum Strategy (Mom)

*a) Construct the return to a long-short momentum strategy portfolio. To that effect every month sort stocks into deciles based on their 1-month lagged 11-month return (that is in month $t$ rank stocks based on their $t$−12 to $t$−1 cumulative return). <br></br> Then compute monthly returns for 10 decile portfolios that equal weight all stocks in each decile. Plot the average annualized portfolio mean, standard deviation, and Sharpe ratios across the 10 deciles in three bar plots. Repeat for value-weighted decile portfolios. Is the evidence consistent with the CAPM?*

In [5]:
# Load the data
data = pd.read_parquet('../data/stock_data.parquet')
data.head()

,permno,date,Rn,shrout,prc,siccd,Rm,rf,mcap,mcap_l,Rn_f,const,Rn_e,Rm_e,w_m
5,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,-0.037172,0.000013,43871.6600,44918.3000,-0.000656,1,-0.018945,-0.037185,0.000005
4,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,0.034744,0.000061,43646.6324,43871.6600,0.020643,1,-0.000717,0.034683,0.000005
3,10001,2010-03-31,0.020643,4361.0,10.1700,4925,0.063668,0.000112,44351.3700,43646.6324,0.124385,1,0.020531,0.063556,0.000005
2,10001,2010-04-30,0.124385,6070.0,11.3900,4925,0.020036,0.000118,69137.3000,44351.3700,0.004829,1,0.124267,0.019918,0.000004
1,10001,2010-05-28,0.004829,6071.0,11.4000,4925,-0.079240,0.000114,69209.4000,69137.3000,-0.043421,1,0.004715,-0.079354,0.000007


In [8]:
# Ensure date is in datetime format
data['date'] = pd.to_datetime(data['date'])

# Step 1: Calculate cumulative returns for each stock over the past 11 months
data['cum_Rn'] = data.groupby('permno')['Rn'].rolling(11).apply(lambda x: np.prod(1 + x) - 1).reset_index(level=0, drop=True)

# Step 2: Sort stocks into deciles based on cumulative return
data['decile'] = data.groupby('date')['cum_Rn'].transform(
    lambda x: pd.qcut(x, 10, labels=False) + 1)

# Step 3: Compute equal-weighted and value-weighted returns for each decile
equal_weighted_returns = data.groupby(['date', 'decile'])['Rn'].mean().unstack()
value_weighted_returns = data.groupby(['date', 'decile']).apply(lambda x: np.average(x['Rn'], weights=x['mcap'])).unstack()

# Calculate annualized mean, standard deviation, and Sharpe ratio
equal_weighted_metrics = equal_weighted_returns.apply(annualized_metrics, axis=0)
value_weighted_metrics = value_weighted_returns.apply(annualized_metrics, axis=0)

# Plotting the metrics
def plot_metrics(metrics, title):
    metrics_df = pd.DataFrame(metrics.tolist(), index=metrics.index, columns=['Mean', 'Std Dev', 'Sharpe Ratio'])
    metrics_df.plot(kind='bar', subplots=True, layout=(3, 1), figsize=(10, 8), title=title)
    plt.tight_layout()
    plt.show()

plot_metrics(equal_weighted_metrics, 'Equal-Weighted Portfolio Metrics')
plot_metrics(value_weighted_metrics, 'Value-Weighted Portfolio Metrics')

# Step 4: Long-Short Momentum Portfolio
long_short_equal = equal_weighted_returns.loc[:, 8:10].mean(axis=1) - equal_weighted_returns.loc[:, 0:2].mean(axis=1)
long_short_value = value_weighted_returns.loc[:, 8:10].mean(axis=1) - value_weighted_returns.loc[:, 0:2].mean(axis=1)

# Step 5: Statistical significance testing
t_stat_equal, p_value_equal = stats.ttest_1samp(long_short_equal, 0)
t_stat_value, p_value_value = stats.ttest_1samp(long_short_value, 0)

# Print the results
print(f"Equal-Weighted Long-Short Strategy: t-stat={t_stat_equal}, p-value={p_value_equal}")
print(f"Value-Weighted Long-Short Strategy: t-stat={t_stat_value}, p-value={p_value_value}")

# Plot long-short strategy metrics
def plot_strategy_metrics(strategy_returns, title):
    mean_return, std_dev, sharpe_ratio = annualized_metrics(strategy_returns)
    metrics_df = pd.DataFrame({
        'Metric': ['Mean Return', 'Standard Deviation', 'Sharpe Ratio'],
        'Value': [mean_return, std_dev, sharpe_ratio]
    })
    metrics_df.plot(kind='bar', x='Metric', y='Value', legend=False, title=title)
    plt.tight_layout()
    plt.show()

plot_strategy_metrics(long_short_equal, 'Equal-Weighted Long-Short Strategy Metrics')
plot_strategy_metrics(long_short_value, 'Value-Weighted Long-Short Strategy Metrics')

KeyboardInterrupt: 